In [2]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, LpAffineExpression

# 加载数据
data = pd.read_csv('/Users/siqinlilv/Desktop/Energy and Transport Analytics/filtered_data_new.csv')  # 更改为你的数据文件路径
data_2022 = data[data['Year'] == 2022]

# 按交叉口、方向和小时分组，确保包含所有必要的车辆类型列
grouped_data = data_2022.groupby(['Count_point_id', 'Direction_of_travel', 'hour']).agg({
    'All_motor_vehicles': 'sum',
    'Pedal_cycles': 'sum',
    'Two_wheeled_motor_vehicles': 'sum',
    'Cars_and_taxis': 'sum',
    'Buses_and_coaches': 'sum',
    'LGVs': 'sum',
    'All_HGVs': 'sum',
}).reset_index()
peak_hours = grouped_data[grouped_data['hour'].isin([7, 8, 9, 17, 18, 19, 20])]

# 分开计算进入和离开的流量
inbound = peak_hours[peak_hours['Direction_of_travel'].isin(['N', 'E'])]
outbound = peak_hours[peak_hours['Direction_of_travel'].isin(['S', 'W'])]
inbound_traffic = inbound.groupby(['Count_point_id', 'hour'])['All_motor_vehicles'].sum()
outbound_traffic = outbound.groupby(['Count_point_id', 'hour'])['All_motor_vehicles'].sum()

# 创建优化模型
model = LpProblem("Traffic_Light_Optimization", LpMinimize)

# 定义碳排放系数
emission_factors = {
    'Pedal_cycles': 0,
    'Two_wheeled_motor_vehicles': 46,
    'Cars_and_taxis': 9,
    'Buses_and_coaches': 32,
    'LGVs': 9,
    'All_HGVs': 32
}

# 决策变量：绿灯时间
g_ij = {(i, j): LpVariable(f"GreenTime_{i}_{j}", lowBound=7, upBound=120, cat='Integer')
        for i in peak_hours['Count_point_id'].unique() for j in peak_hours['hour'].unique()}

# 更新目标函数以考虑交通流量和碳排放的权衡
traffic_weight = 0.5  # 交通流量的权重
emission_weight = 0.5  # 碳排放的权重
traffic_emission_dict = {
    (row['Count_point_id'], row['hour']): (
        traffic_weight * row['All_motor_vehicles'],
        emission_weight * sum(row[vehicle] * emission_factors[vehicle] for vehicle in emission_factors if vehicle in row)
    )
    for index, row in peak_hours.iterrows()
}
model += lpSum([
    g_ij[(i, j)] * (traffic - emission)
    for (i, j), (traffic, emission) in traffic_emission_dict.items()
])

# 约束条件

for i in peak_hours['Count_point_id'].unique():
    for j in peak_hours['hour'].unique():
        model += g_ij[(i, j)] >= 7
        model += g_ij[(i, j)] <= 120
        # 总绿灯时间约束
        model += lpSum(g_ij[(i, j)] for j in peak_hours['hour'].unique() if (i, j) in g_ij) <= 300
        # 流量守恒约束
        inflow = inbound_traffic.get((i, j), 0)
        outflow = outbound_traffic.get((i, j), 0)
        # 使用 LpAffineExpression 添加等式约束
        model += (LpAffineExpression([(g_ij[(i, j)], 1)]) == inflow - outflow,
                  f"Flow_Conservation_{i}_{j}")



# 解决问题
model.solve()

# 输出结果
print("Status:", LpStatus[model.status])
for (i, j) in g_ij:
    print(f"Intersection {i} during hour {j}: Green time = {g_ij[(i, j)].value()} seconds")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/siqinlilv/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/d3/mnnr4rtn1zb5wkz6xf4ytftc0000gn/T/cb26d90e45a94a4eb79de21fbdc63c58-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/d3/mnnr4rtn1zb5wkz6xf4ytftc0000gn/T/cb26d90e45a94a4eb79de21fbdc63c58-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10105 COLUMNS
At line 37868 RHS
At line 47969 BOUNDS
At line 53020 ENDATA
Problem MODEL has 10100 rows, 2525 columns and 20200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.09 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.13   (Wallclock seconds):       0.17

Status: Infeasible
Intersection 6001 during hour 7: Green time = 120.0 seconds
Intersection 6001 during hour 8: Green time = 120.